In [1]:
#importing all relavent libraries, not some are not used 
import os
import h5py
import numpy as np
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import h5py
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
import scipy.signal as sig
import matplotlib.pyplot as plt
import time

In [2]:
#location of training files, toy files are provided on the repo.
data_directory = "" #use if training files are in the same folder as this code 
signals = np.load(data_directory + "./psd_data.npy") #includes PSD graphs

In [3]:
noise_amplitude = 1e-22
#each data segment corresponds to sampling frequency 100Hz for 30 seconds
num_datapoints = int(30*100)
gaussian_noise = np.random.normal(0, noise_amplitude, (len(signals), num_datapoints))

psd_noise = sig.welch(gaussian_noise, fs = 100)[1][:, :20]
#normalize as with previous
noises = np.log10(psd_noise)
assert(len(noises) == len(signals))

In [4]:
data = np.vstack([signals, noises])
labels = np.hstack([np.ones( (len(signals), ) ), np.zeros( (len(noises), ) )])

#shuffle in unison
p = np.random.permutation(len(data))
data = data[p]
labels_s = labels[p]
params = np.expand_dims(labels_s, axis = 1) #formatting for neural network

In [5]:
#normalizing by average value, make sure to maintain this value for pipeline code
print("avg val,", np.average(data))
data -= np.average(data)

avg val, -45.579468922452655


In [6]:
#neural network hyperparameters
batch_size = 20
input_shape = (20,)
epochs = 5

In [7]:
split = 95000
test_split = split
x_train = data[:split, :]
x_test = data[test_split:, :]

y_train = params[:split, :]
y_test = params[test_split:, :]

print("{}:xtrain, {}:xtest, {}:ytrain, {}:ytest".
      format(x_train.shape, x_test.shape, y_train.shape,y_test.shape))

(95000, 20):xtrain, (5000, 20):xtest, (95000, 1):ytrain, (5000, 1):ytest


In [8]:
#define neural network
model = Sequential()

model.add(Dense(1024, activation = 'relu', 
                input_shape = input_shape))
model.add(Dense(1024, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(1, activation = "sigmoid"))

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              21504     
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_2 (Dense)             (None, 512)               524800    
                                                                 
 dense_3 (Dense)             (None, 512)               262656    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 512)               262656    
                                                                 
 dense_5 (Dense)             (None, 1)                 5

In [9]:
#compile model with relevant loss, optimizer, and metric
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

#fit to the training data
history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

#evaluate the preformance on the testing set
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/5
4750/4750 [==============================] - 31s 6ms/step - loss: 5.6150e-04 - accuracy: 0.9998 - val_loss: 6.1770e-34 - val_accuracy: 1.0000
Epoch 2/5
4750/4750 [==============================] - 30s 6ms/step - loss: 5.2729e-15 - accuracy: 1.0000 - val_loss: 6.1770e-34 - val_accuracy: 1.0000
Epoch 3/5
4750/4750 [==============================] - 32s 7ms/step - loss: 1.6867e-16 - accuracy: 1.0000 - val_loss: 6.1770e-34 - val_accuracy: 1.0000
Epoch 4/5
4750/4750 [==============================] - 32s 7ms/step - loss: 1.6121e-14 - accuracy: 1.0000 - val_loss: 6.1460e-34 - val_accuracy: 1.0000
Epoch 5/5
4750/4750 [==============================] - 32s 7ms/step - loss: 3.9046e-14 - accuracy: 1.0000 - val_loss: 5.9048e-34 - val_accuracy: 1.0000
Test loss: 5.904803375300498e-34
Test accuracy: 1.0


In [ ]:
#save the model to relevant location, will need this for the pipeline simulation
model_save_directory = "" #save model to same the same folder as this program
model.save("./binary_classification_network.h5", include_optimizer = False)
